In [1]:
import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  NVIDIA GeForce GTX 1650
Using device: cuda


In [5]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit

# Configuration
SOURCE_ROOT = r"C:\NPersonal\Projects\SDP\Prediction Stuff\Dataset\MDVR Spectrogram Dataset"
DEST_ROOT = r"C:\NPersonal\Projects\SDP\Prediction Stuff\Dataset\MDVR"
SPLIT_RATIO = (0.7, 0.15, 0.15)  # Train, Val, Test
SEED = 42

def extract_patient_number(folder_name):
    """Extract base patient number from complex folder names"""
    # Example: "S_ID00_hc_0_0_0" -> "ID00"
    # Example: "R_PD_123_1_2" -> "PD_123"
    parts = folder_name.split('_')
    if parts[0] in ['S', 'R']:
        base_parts = parts[1:-3]  # Remove S/R prefix and numeric suffixes
    else:
        base_parts = parts[:-3]  # For non-S/R prefixed patients
    return '_'.join(base_parts)

def process_dataset():
    # Collect all patients with their base numbers
    patients = []
    
    for class_name in ['HC', 'PD']:
        class_path = os.path.join(SOURCE_ROOT, class_name)
        if not os.path.exists(class_path):
            continue
            
        for folder in os.listdir(class_path):
            folder_path = os.path.join(class_path, folder)
            if os.path.isdir(folder_path):
                base_number = extract_patient_number(folder)
                patients.append({
                    'original_path': folder_path,
                    'class': class_name,
                    'base_number': base_number,
                    'full_name': folder
                })

    # Create DataFrame and group by base number
    df = pd.DataFrame(patients)
    grouped = df.groupby(['base_number', 'class'])

    # Prepare data for splitting
    unique_patients = []
    for (base_num, cls), group in grouped:
        unique_patients.append({
            'base_number': base_num,
            'class': cls,
            'paths': group['original_path'].tolist(),
            'count': len(group)
        })

    # Convert to DataFrame for splitting
    split_df = pd.DataFrame(unique_patients)

    # Split patients into train/val/test
    gss = GroupShuffleSplit(n_splits=1, test_size=SPLIT_RATIO[1]+SPLIT_RATIO[2], random_state=SEED)
    train_idx, temp_idx = next(gss.split(split_df, groups=split_df['base_number']))

    gss_val_test = GroupShuffleSplit(n_splits=1, test_size=SPLIT_RATIO[2]/(SPLIT_RATIO[1]+SPLIT_RATIO[2]), random_state=SEED)
    val_idx, test_idx = next(gss_val_test.split(split_df.iloc[temp_idx], groups=split_df.iloc[temp_idx]['base_number']))

    # Create splits
    splits = {
        'train': split_df.iloc[train_idx],
        'val': split_df.iloc[temp_idx].iloc[val_idx],
        'test': split_df.iloc[temp_idx].iloc[test_idx]
    }

    # Copy files while preserving S/R versions
    for split_name, split_data in splits.items():
        print(f"Processing {split_name} set...")
        for _, patient in split_data.iterrows():
            for version_path in patient['paths']:
                # Create destination path
                dest_folder = os.path.join(
                    DEST_ROOT,
                    split_name,
                    patient['class'],
                    os.path.basename(version_path)  # Keep original folder name
                )
                
                os.makedirs(dest_folder, exist_ok=True)
                
                # Copy all PNG files
                for file in os.listdir(version_path):
                    if file.lower().endswith('.png'):
                        src = os.path.join(version_path, file)
                        dst = os.path.join(dest_folder, file)
                        if not os.path.exists(dst):
                            shutil.copy2(src, dst)

    # Verification
    print("\n📊 Final Verification:")
    for split in ['train', 'val', 'test']:
        split_path = os.path.join(DEST_ROOT, split)
        total = 0
        for root, dirs, files in os.walk(split_path):
            total += len(files)
        print(f"{split.upper()} - {total} images")
        print(f"Sample patient folders: {os.listdir(split_path)[:2]}")

if __name__ == "__main__":
    process_dataset()
    print("\n✅ Dataset organized with strict patient-level splits!")


Processing train set...
Processing val set...
Processing test set...

📊 Final Verification:
TRAIN - 41744 images
Sample patient folders: ['HC', 'PD']
VAL - 13509 images
Sample patient folders: ['HC', 'PD']
TEST - 8929 images
Sample patient folders: ['HC', 'PD']

✅ Dataset organized with strict patient-level splits!


In [11]:
# import os
# import torch
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms
# from PIL import Image

# # Configuration
# DATA_ROOT = r"C:\NPersonal\Projects\SDP\Prediction Stuff\Dataset\MDVR"
# BATCH_SIZE = 32
# DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # Custom Dataset Class
# class ParkinsonSpectrogramDataset(Dataset):
#     def __init__(self, root_dir, transform=None):
#         self.root_dir = root_dir
#         self.transform = transform
#         self.samples = self._load_samples()
        
#     def _load_samples(self):
#         samples = []
#         for category in ["ReadText", "SpontaneousDialogue"]:
#             for class_name in ['HC', 'PD']:
#                 class_dir = os.path.join(self.root_dir, category, class_name)
#                 if not os.path.exists(class_dir):
#                     continue
            
#                 # Traverse subfolders inside HC/PD
#                 for patient_folder in os.listdir(class_dir):
#                     patient_path = os.path.join(class_dir, patient_folder)
#                     if os.path.isdir(patient_path):  # Ensure it's a directory
#                         for img_file in os.listdir(patient_path):
#                             if img_file.lower().endswith('.png'):  # Only PNG images
#                                 img_path = os.path.join(patient_path, img_file)
#                                 samples.append((img_path, 0 if class_name == 'HC' else 1))

#         print(f"✅ Loaded {len(samples)} samples from {self.root_dir}")  # Debugging print
#         return samples

    
#     def __len__(self):
#         return len(self.samples)
    
#     def __getitem__(self, idx):
#         img_path, label = self.samples[idx]
#         img = Image.open(img_path).convert('L')  # Convert to grayscale
#         if self.transform:
#             img = self.transform(img)
#         return img, label

# # Data Transforms
# train_transform = transforms.Compose([
#     transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Only slight translations
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize between -1 and 1
# ])

# test_transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5], std=[0.5])
# ])

# # Create Datasets
# train_dataset = ParkinsonSpectrogramDataset(
#     os.path.join(DATA_ROOT, 'train'),
#     transform=train_transform
# )

# val_dataset = ParkinsonSpectrogramDataset(
#     os.path.join(DATA_ROOT, 'val'),
#     transform=test_transform
# )

# test_dataset = ParkinsonSpectrogramDataset(
#     os.path.join(DATA_ROOT, 'test'),
#     transform=test_transform
# )

# # Create DataLoaders
# # train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
# # val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=4)
# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
# val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

# test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=4)

# print("✅ DataLoaders ready!")
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {DEVICE}")  # Should print: Using device: cuda


import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# Configuration
DATA_ROOT = r"C:\NPersonal\Projects\SDP\Prediction Stuff\Dataset\MDVR"
BATCH_SIZE = 32
NUM_WORKERS = 0 # Optimize based on CPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset Class
class ParkinsonSpectrogramDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = self._load_samples()
        
    def _load_samples(self):
        samples = []
        for category in ["ReadText", "SpontaneousDialogue"]:
            for class_name in ['HC', 'PD']:
                class_dir = os.path.join(self.root_dir, category, class_name)
                if not os.path.exists(class_dir):
                    continue
            
                for patient_folder in os.listdir(class_dir):
                    patient_path = os.path.join(class_dir, patient_folder)
                    if os.path.isdir(patient_path):  # Ensure it's a directory
                        for img_file in os.listdir(patient_path):
                            if img_file.lower().endswith('.png'):  # Only PNG images
                                img_path = os.path.join(patient_path, img_file)
                                samples.append((img_path, 0 if class_name == 'HC' else 1))

        print(f"✅ Loaded {len(samples)} samples from {self.root_dir}")
        return samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert('L')  # Convert to grayscale
        if self.transform:
            img = self.transform(img)
        return img, label

# Data Transforms
train_transform = transforms.Compose([
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Only slight translations
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize between -1 and 1
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Create Datasets
train_dataset = ParkinsonSpectrogramDataset(os.path.join(DATA_ROOT, 'train'), transform=train_transform)
val_dataset = ParkinsonSpectrogramDataset(os.path.join(DATA_ROOT, 'val'), transform=test_transform)
test_dataset = ParkinsonSpectrogramDataset(os.path.join(DATA_ROOT, 'test'), transform=test_transform)

# Optimized DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                          # num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True)
                          num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=False)

val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                        # num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True)
                        num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=False)

test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                         num_workers=NUM_WORKERS, pin_memory=True)

print(f"✅ DataLoaders ready! Using device: {DEVICE}")


✅ Loaded 47439 samples from C:\NPersonal\Projects\SDP\Prediction Stuff\Dataset\MDVR\train
✅ Loaded 7890 samples from C:\NPersonal\Projects\SDP\Prediction Stuff\Dataset\MDVR\val
✅ Loaded 8853 samples from C:\NPersonal\Projects\SDP\Prediction Stuff\Dataset\MDVR\test
✅ DataLoaders ready! Using device: cuda


In [12]:
print(f"Number of batches in train_loader: {len(train_loader)}")

print(f"Dataset size: {len(train_dataset)}")
print(f"Number of batches: {len(train_loader)}")

sample, label = train_dataset[0]  # Try loading one sample
print(sample.shape, label)



Number of batches in train_loader: 1483
Dataset size: 47439
Number of batches: 1483
torch.Size([1, 200, 496]) 0


In [13]:
for batch in train_loader:
    print(batch)  # Check if batches are being created
    break  # Print only the first batch


[tensor([[[[-0.3647, -0.3647, -0.3647,  ..., -1.0000, -1.0000, -1.0000],
          [-0.3647, -0.3647, -0.3647,  ..., -1.0000, -1.0000, -1.0000],
          [-0.4353, -0.4353, -0.4353,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]]],


        [[[ 0.3255,  0.3255,  0.3255,  ..., -1.0000, -1.0000, -1.0000],
          [ 0.3255,  0.3255,  0.3255,  ..., -1.0000, -1.0000, -1.0000],
          [ 0.5686,  0.5686,  0.5686,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]]],


        [[[-1.0000, -1.0000, -1.0000,  ..., -0.8196, -0.8196, -0.8196],
          [-1.0000, -1.00

In [14]:
# STRUCTURE 1

# import torch
# import torch.nn as nn

# class ParkinsonCNN(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.features = nn.Sequential(
#             nn.Conv2d(1, 32, kernel_size=3, padding=1),
#             nn.BatchNorm2d(32),
#             nn.ReLU(),
#             nn.MaxPool2d(2),  # 32x248x100
            
#             nn.Conv2d(32, 64, kernel_size=3, padding=1),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
#             nn.MaxPool2d(2),  # 64x124x50
            
#             nn.Conv2d(64, 128, kernel_size=3, padding=1),
#             nn.BatchNorm2d(128),
#             nn.ReLU(),
#             nn.MaxPool2d(2),  # 128x62x25

#             nn.Conv2d(128, 256, kernel_size=3, padding=1),
#             nn.BatchNorm2d(256),
#             nn.ReLU(),
#             nn.MaxPool2d(2),  # 256x31x12
#         )
        
#         self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))  
        
#         self.classifier = nn.Sequential(
#             nn.Flatten(),
#             nn.Linear(256, 128),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Linear(128, 2)  # Binary classification (HC vs PD)
#         )
        
#     def forward(self, x):
#         x = self.features(x)
#         x = self.global_avg_pool(x)
#         return self.classifier(x)

# # Initialize Model
# model = ParkinsonCNN().to(DEVICE)


# STRUCTURE 2 (CLAUDE)
import torch
import torch.nn as nn
import torch.nn.functional as F

class ImprovedParkinsonCNN(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super().__init__()
        # First convolutional block
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.2)
        )
        
        # Second convolutional block
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.3)
        )
        
        # Third convolutional block
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.4)
        )
        
        # Fourth convolutional block
        self.conv4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.4)
        )
        
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(64, 2)  # Binary classification
        )
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.global_avg_pool(x)
        x = self.classifier(x)
        return x

# Enhanced data augmentation
def get_enhanced_transforms():
    from torchvision import transforms
    
    train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomAffine(degrees=5, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])
    
    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])
    
    return train_transform, test_transform




In [10]:
# import torch.optim as optim
# import torch
# from tqdm import tqdm  # Progress bar for training
# import torch.nn as nn

# def train_model(model, criterion, optimizer, scheduler, num_epochs=50):
#     model.to(DEVICE)  # Move model to device
#     best_acc = 0.0
#     history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
    
#     for epoch in range(num_epochs):
#         model.train()
#         running_loss = 0.0
#         correct = 0
#         total = 0
        
#         # Batch progress bar
#         train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)
        
#         for batch_idx, (inputs, labels) in enumerate(train_loader_tqdm):
#             inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

#             optimizer.zero_grad()
#             outputs = model(inputs)  # Raw logits (no softmax needed)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
            
#             running_loss += loss.item()
#             _, predicted = outputs.max(1)
#             total += labels.size(0)
#             correct += predicted.eq(labels).sum().item()

#             # Print batch update every 10 batches
#             if (batch_idx + 1) % 10 == 0:
#                 avg_loss = running_loss / (batch_idx + 1)
#                 acc = correct / total
#                 train_loader_tqdm.set_postfix(loss=avg_loss, acc=acc)

#         train_loss = running_loss / len(train_loader)
#         train_acc = correct / total

#         # Validation
#         val_loss, val_acc = evaluate(model, criterion, val_loader)

#         # Step LR Scheduler
#         scheduler.step(val_loss)
        
#         history['train_loss'].append(train_loss)
#         history['val_loss'].append(val_loss)
#         history['train_acc'].append(train_acc)
#         history['val_acc'].append(val_acc)
        
#         print(f"\nEpoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
#         print(f"Validation Loss: {val_loss:.4f}, Validation Acc: {val_acc:.4f}\n")
        
#         # Save best model
#         if val_acc > best_acc:
#             best_acc = val_acc
#             torch.save({
#                 'model_state_dict': model.state_dict(),
#                 'optimizer_state_dict': optimizer.state_dict(),
#                 'epoch': epoch,
#                 'val_acc': val_acc
#             }, 'best_model.pth')
            
#     return history

# def evaluate(model, criterion, loader):
#     model.to(DEVICE)  # Ensure model is on correct device
#     model.eval()
#     running_loss = 0.0
#     correct = 0
#     total = 0
    
#     with torch.no_grad():
#         for inputs, labels in loader:
#             inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
            
#             running_loss += loss.item()
#             _, predicted = outputs.max(1)
#             total += labels.size(0)
#             correct += predicted.eq(labels).sum().item()
            
#     return running_loss / len(loader), correct / total

# # Initialize training
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # L2 regularization
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# history = train_model(model, criterion, optimizer, scheduler)

import torch.optim as optim
import torch
from tqdm import tqdm  # Progress bar for training
import torch.nn as nn

# Ensure CUDA is used optimally
torch.backends.cudnn.benchmark = True

# Check GPU availability
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

def train_model(model, criterion, optimizer, scheduler, num_epochs=50):
    model.to(DEVICE)  # Move model to device
    print(next(model.parameters()).device)  # Confirm model is on GPU
    
    best_acc = 0.0
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)
        
        for batch_idx, (inputs, labels) in enumerate(train_loader_tqdm):
            inputs, labels = inputs.to(DEVICE, non_blocking=True), labels.to(DEVICE, non_blocking=True)

            optimizer.zero_grad()
            outputs = model(inputs)  # Raw logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            # Print batch update every 10 batches
            if (batch_idx + 1) % 10 == 0:
                avg_loss = running_loss / (batch_idx + 1)
                acc = correct / total
                train_loader_tqdm.set_postfix(loss=avg_loss, acc=acc)

        train_loss = running_loss / len(train_loader)
        train_acc = correct / total

        # Validation
        val_loss, val_acc = evaluate(model, criterion, val_loader)

        # Step LR Scheduler
        scheduler.step(val_loss)
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        
        print(f"\nEpoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"Validation Loss: {val_loss:.4f}, Validation Acc: {val_acc:.4f}\n")
        
        # Save best model
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'epoch': epoch,
                'val_acc': val_acc
            }, 'best_model.pth')
            
    return history

def evaluate(model, criterion, loader):
    model.to(DEVICE)  # Ensure model is on correct device
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE, non_blocking=True), labels.to(DEVICE, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
    return running_loss / len(loader), correct / total

# Initialize training
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

history = train_model(model, criterion, optimizer, scheduler)


Model is on: cuda:0


Epoch 1/50: 100%|███████████████████████████████████████████| 1483/1483 [08:10<00:00,  3.03it/s, acc=0.701, loss=0.572]



Epoch 1/50 - Train Loss: 0.5719, Train Acc: 0.7014
Validation Loss: 0.6050, Validation Acc: 0.6782



Epoch 2/50: 100%|███████████████████████████████████████████| 1483/1483 [07:42<00:00,  3.20it/s, acc=0.709, loss=0.559]



Epoch 2/50 - Train Loss: 0.5586, Train Acc: 0.7086
Validation Loss: 0.6528, Validation Acc: 0.6465



...

Epoch 12/50: 100%|██████████████████████████████████████████| 1483/1483 [07:26<00:00,  3.32it/s, acc=0.737, loss=0.511]



Epoch 12/50 - Train Loss: 0.5108, Train Acc: 0.7367
Validation Loss: 0.5765, Validation Acc: 0.6864



Epoch 13/50: 100%|██████████████████████████████████████████| 1483/1483 [07:28<00:00,  3.31it/s, acc=0.742, loss=0.506]



Epoch 13/50 - Train Loss: 0.5055, Train Acc: 0.7420
Validation Loss: 0.5904, Validation Acc: 0.6766



Epoch 14/50:  91%|██████████████████████████████████████▎   | 1353/1483 [06:53<00:39,  3.27it/s, acc=0.742, loss=0.505]

KeyboardInterrupt



In [ ]:
import torch
import os
from sklearn.metrics import classification_report, roc_auc_score

# Ensure model is defined
model = ParkinsonCNN().to(DEVICE)

# Check if checkpoint exists
if not os.path.exists('best_model.pth'):
    raise FileNotFoundError("Model checkpoint 'best_model.pth' not found.")

# Load best model
checkpoint = torch.load('best_model.pth', map_location=DEVICE)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)  # Ensure model is on the right device

# Define test function
def test_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_preds, all_labels

# Evaluate on test set
test_preds, test_labels = test_model(model, test_loader)

# Print classification report
print("Test Performance:")
print(classification_report(test_labels, test_preds))

# Handle AUC-ROC edge case
try:
    auc_score = roc_auc_score(test_labels, test_preds)
except ValueError:
    auc_score = "Undefined (Only one class present in test set)"
print(f"AUC-ROC: {auc_score}")


In [ ]:
import matplotlib.pyplot as plt

def plot_training(history):
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.title('Loss Curve')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['val_acc'], label='Val Accuracy')
    plt.title('Accuracy Curve')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_curves.png')
    plt.show()

plot_training(history)